# 4.1 The Metacircular Evaluator

In [1]:
(define (eval exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((quoted? exp)
         (text-of-quotation exp))
        ;; assignment/definition recursively call eval to comput
        ;; new value associated with a variable
        ((assignment? exp)
         (eval-assignment exp env))
        ((definition? exp)
         (eval-definition exp env))
        ((if? exp)
         (eval-if exp env))
        ;; transform into applicable procedure within environment
        ((lambda? exp)
         (eval-lambda exp env))
        ;; evaluate sequence of expressions in order
        ((begin? exp)
         (eval-begin exp env))
        ;; case analysis -> nest of if expressions which are then evaluated
        ((cond? exp)
         (eval-cond exp env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

(define (apply procedure arguments)
  (cond ((primitive-procedure? procedure)
         (apply-primitive-procedure
          procedure
          arguments))
        ;; sequentially eval each expression in procedure body
        ;; extend base environment to include new frame binding
        ;; params of procedure to the arguments it is applied to
        ((compound-procedure? procedure)
         (eval-sequence
          (procedure-body procedure)
          (extend-environment
           (procedure-parameters
            procedure)
           arguments
           (procedure-environment
            procedure))))
        (else
         (error "Unkown procedure
                type: APPLY"
                procedure))))

;; produce list of arguments to which a procedure is to be applied
;; evaluate each operand -> return list of results
(define (list-of-values exps env)
  (if (no-operands? exps)
      '()
      (cons (eval (first-operand exps) env)
            (list-of-values
             (rest-operands exps)
             env))))

(define (eval-if exp env)
  (if (true? (eval (if-predicate exp) env))
      (eval (if-consequent exp) env)
      (eval (if-alternative exp) env)))

;; eval a sequence of expressions in order
;; return value of final expression
(define (eval-sequence exps env)
  (cond ((last-exp? exps)
         (eval (first-exp exps) env))
        (else
         (eval (first-exp exps) env)
         (eval-sequence (rest-exps exps)
                        env))))

;; assign value to variable
(define (eval-assignment exp env)
  (set-variable-value!
   (assignment-variable exp)
   (eval (assignment-value exp) env)
   env)
  'ok)

;; define a variable
(define (eval-definition exp env)
  (define-variable!
   (definition-variable exp)
   (eval (definition-value exp) env)
   env)
  'ok)

(define (eval-begin exp env)
  (eval-sequence
   (begin-actions exp)
   env))

(define (eval-cond exp env)
  (eval (cond->if exp) env))

(define (eval-lambda exp env)
  (make-procedure
   (lambda-parameters exp)
   (lambda-body exp)
   env))


;; expressions

(define (self-evaluating? exp)
  (cond ((number? exp) #t)
        ((string? exp) #t)
        (else #f)))

(define (variable? exp) (symbol? exp))

;; quotations: (quote <text-of-quotation>)
(define (quoted? exp)
  (tagged-list? exp 'quote))
(define (text-of-quotation exp)
  (cadr exp))

(define (tagged-list? exp tag)
  (if (pair? exp)
      (eq? (car exp) tag)
      false))

;; assignments: (set! <var> <value>)
(define (assignment? exp)
  (tagged-list? exp 'set!))
(define (assignment-variable exp)
  (cadr exp))
(define (assignment-value exp) (caddr exp))

;; definitions: (define <var> <value>)
;; OR (define (<var> <param-1> ... <param-n>) <body>)
(define (definition? exp)
  (tagged-list? exp 'define))
(define (definition-variable exp)
  (if (symbol? (cadr exp))
      (cadr exp)
      (caadr exp)))
(define (definition-value exp)
  (if (symbol? (cadr exp))
      (caddr exp)
      (make-lambda
       (cdadr exp) ; formal parameters
       (cddr exp)))) ; body

;; lambda expressions: (lambda (<param-1> ... <param-n>) <body>)
(define (lambda? exp)
  (tagged-list? exp 'lambda))
(define (lambda-parameters exp) (cadr exp))
(define (lambda-body exp) (cddr exp))
(define (make-lambda parameters body)
  (cons 'lambda (cons parameters body)))

;; conditionals: (if <predicate> <consequent> <alternative>?)
;; alternative is optional -> default to false
(define (if? exp) (tagged-list? exp 'if))
(define (if-predicate exp) (cadr exp))
(define (if-consequent exp) (caddr exp))
(define (if-alternative exp)
  (if (not (null? (cdddr exp)))
      (cadddr exp)
      'false)) ;; default to false if no alternative
(define (make-if predicate
                 consequent
                 alternative)
  (list 'if
        predicate
        consequent
        alternative))

;; begin: (begin <exp-1> ... <exp-n>)
;; package a sequence of expressions into a single expression
(define (begin? exp)
  (tagged-list? exp 'begin))
(define (begin-actions exp) (cdr exp))
(define (last-exp? seq) (null? (cdr seq)))
(define (first-exp seq) (car seq))
(define (rest-exps seq) (cdr seq))
(define (sequence->exp seq)
  (cond ((null? seq) seq)
        ((last-exp? seq) (first-exp seq))
        (else (make-begin seq))))
(define (make-begin seq) (cons 'begin seq))

;; procedure application: any compound expression not defined as an expression type above
;; (<operator> <operand-1> ... <operand-n>)
(define (application? exp) (pair? exp))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))
(define (no-operands? ops) (null? ops))
(define (first-operand ops) (car ops))
(define (rest-operands ops) (cdr ops))

;; cond: (cond ((<predicate-1> <result-1> ... <predicate-n> <result-1n) (else <result>))
;; implemented as nested if expressions
(define (cond? exp)
  (tagged-list? exp 'cond))
(define (cond-clauses exp) (cdr exp))
(define (cond-else-clause? clause)
  (eq? (cond-predicate clause) 'else))
(define (cond-predicate clause)
  (car clause))
(define (cond-actions clause)
  (cdr clause))
(define (cond->if exp)
  (expand-clauses (cond-clauses exp)))
(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (make-if (cond-predicate first)
                     (sequence->exp
                      (cond-actions first))
                     (expand-clauses
                      rest))))))

## 4.1
Nested `let`s will force the operands to be evaluated in the desired order. The `let` must be nested as multiple assignments in a single let will use the underlying Scheme order of evaluation:
```scheme
(let ((foo exp1) ((bar exp2)))
  ...)
```
Is equivalent to:
```scheme
((lambda (foo bar)...) exp1 exp2)
```

In [2]:
(define (list-of-values-lr exps env)
  (if (no-operands? exps)
      '()
      (let ((left (eval (first-operand exps) env)))
        (let ((right (list-of-values-lr (rest-operands exps) env)))
          (cons left right)))))

(define (list-of-values-rl exps env)
  (if (no-operands? exps)
      '()
      (let ((right (list-of-values-lr (rest-operands exps) env)))
        (let ((left (eval (first-operand exps) env)))
          (cons right left)))))

## 4.2
1. Moving the clause for procedure applications to before the clause for assignments produces the following for `(define x 3)`:
```scheme
(eval '(define x 3) env)
(application? '(define x 3)) => true
(apply (eval 'define env) (list-of-values ('x 3) env))
(eval 'define env)
(variable? 'define) => true
(lookup-variable-value 'define env) ERROR
```
2. Changing procedure applcations to start with `call` requires changing the `application?` predicate and `operator`/`operand` selectors:
```scheme
(define (application? exp)
  (tagged-list? exp 'call))
(define (operator exp) (cadr exp))
(define (operand exp) (cddr exp))
```

## 4.3

In [3]:
(define (eval-data-directed exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((get (car exp))
         ((get (car exp)) exp env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

(define (install-eval-data-directed)
  (put 'quote text-of-quotation)
  (put 'set! eval-assignment) 
  (put 'define eval-definition)
  (put 'if eval-if)
  (put 'lambda eval-lambda)
  (put 'begin eval-begin)
  (put 'cond eval-cond)
  'done)

## 4.4
#### `and`
- Expressions evaluated left->right
- Any expression evaluates to false -> return false
    - Remaining expressions **not** evaluated
- All expressions evaluate to true -> return value of final expression
- No expressions -> return true

#### `or`
- Expressions evaluated left->right
- Any expression evalues to true -> return that value
    - Remaining expressions **not** evaluated
- All expressions evaluate to false -> return false
- No expressions -> return false

In [5]:
;; extend to include boolean values
(define (self-evaluating? exp)
  (or (number? exp)
      (string? exp)
      (boolean? exp)))

(define (and? exp)
  (tagged-list? exp 'and))
(define (or? exp)
  (tagged-list? exp 'or))
(define (boolean-exps exp)
  (cdr exp))

(define (eval-and exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env)
           (eval-exps (rest-exps exp)))
          (else #f)))
  (eval-exps (boolean-exps exp)))

(define (eval-or exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env) #t)
          (else (eval-exps (rest-exps exp)))))
  (eval-exps (boolean-exps exp)))


;; OR with derived expressions using nested if
(define (eval-and exp env)
  (eval (and->if (boolean-exps exp) env) env))
(define (eval-or exp env)
  (eval (or->if (boolean-exps exp) env) env))

(define (and->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               (and->if (rest-exps exp) env)
               #f))))

(define (or->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               #t
               (or->if (rest-exps exp) env)))))


## 4.5
Extend `cond` to support `(<test> => <recipient>)` syntax.

In [6]:
(define (cond-arrow-form? clause)
  (eq? (cadr clause) '=>))
(define (cond-recipient clause)
  (caddr clause))

;; ((<predicate-clauses>) (<result-clauses>)) -> (<result-clauses>)
;; ((<test>) => <recipient>) -> (<recipient> (<test>))
(define (cond-consequent clause predicate)
  (if (cond-arrow-form? clause)
      (list (cond-recipient clause) predicate)
      (sequence->exp (cond-actions clause))))

(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (let ((predicate (cond-predicate first)))
              (make-if predicate
                       (cond-consequent first predicate)
                       (expand-clauses rest)))))))